In [10]:
import pandas as pd
from functools import reduce

In [11]:
xl = pd.ExcelFile('physa_microplastics_raw.xlsx')

In [28]:
caps   = xl.parse('TOTAL EGG CAPSULE')
hatch  = xl.parse('HATCHING SUCCESS')
speed  = xl.parse('LOCOMOTION')
reprod = xl.parse('REPRODUCTION')

In [13]:
speed = speed.rename(columns={'no.of days': 'days'})

# Helper function to melt data into long format
def melt_sheet(df, id_vars, value_name):
    long = df.melt(id_vars=id_vars,
                   var_name='group',
                   value_name=value_name)
    long['dose'] = (long['group']
                    .str.extract(r'(\d+)')   # extract 20, 30, etc.
                    .fillna(0)               # control group → 0
                    .astype(int))
    return long.drop(columns='group')

In [14]:
caps_long   = melt_sheet(caps,   id_vars=['days'], value_name='capsules')
speed_long  = melt_sheet(speed,  id_vars=['days'], value_name='speed')
hatch_long  = melt_sheet(hatch,  id_vars=[],       value_name='hatch_ratio')
reprod_long = melt_sheet(reprod, id_vars=[],       value_name='juveniles')

In [15]:
hatch_long['days'] = range(1, len(hatch_long) + 1)
reprod_long['days'] = range(1, len(reprod_long) + 1)

In [16]:
dfs = [caps_long, speed_long, hatch_long, reprod_long]
tidy = reduce(lambda left, right: pd.merge(left, right, on=['dose', 'days'], how='outer'), dfs)

In [17]:
tidy.to_csv('physa_microplastics_tidy.csv', index=False)